In [1]:
import os
from datetime import datetime
import pyspark
from pyspark.sql.functions import date_format,to_date, to_timestamp
from pyspark.sql import SparkSession
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row
import boto3
import logging

In [2]:
AWS_ENDPOINT = "http://minio-webserver:9000"
AWS_ACCESS_KEY_ID = "U6dHDkvTv3CdrviA"
AWS_SECRET_ACCESS_KEY = "o8LTVjrVtNuzOv7DjGOxhN0HQDeksEej"
AWS_REGION ="us-east-1"

BUCKET_STAGE="antaq-stage"
BUCKET_ANALYTICS="antaq-analytics"

SPARK_APP_NAME= "dataviva-notebook-DW"
SPARK_MASTER_URL="spark://spark-master:7077"

clientS3 = boto3.client('s3',
    endpoint_url=AWS_ENDPOINT,
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    aws_session_token=None,
    config=boto3.session.Config(signature_version='s3v4'),
    verify=False,
    region_name=AWS_REGION
)
clientS3

In [3]:
def obterJars():
    dir = "/tmp/spark-jars"
    jars = os.listdir(dir)
    stringJars = ""

    for jar in jars:
        print(f"{dir}/{jar}")
        stringJars += f"{dir}/{jar},"
    return stringJars[:-1]

In [4]:
# client_spark.stop()

conf = pyspark.SparkConf()
conf.setAppName(SPARK_APP_NAME)
conf.setMaster(SPARK_MASTER_URL)

conf.set("spark.hadoop.fs.s3a.endpoint", AWS_ENDPOINT) \
        .set("spark.hadoop.fs.s3a.endpoint.region", AWS_REGION) \
        .set("spark.hadoop.fs.s3a.access.key",AWS_ACCESS_KEY_ID) \
        .set("spark.hadoop.fs.s3a.secret.key", AWS_SECRET_ACCESS_KEY) \
        .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
        .set("spark.hadoop.fs.s3a.connection.ssl.enabled", False) \
        .set("spark.hadoop.com.amazonaws.services.s3.enableV2", True) \
        .set("spark.hadoop.fs.s3a.committer.staging.conflict-mode", "replace") \
        .set("spark.hadoop.fs.s3a.fast.upload", True) \
        .set("spark.hadoop.fs.s3a.path.style.access", True) \
        .set("spark.sql.sources.commitProtocolClass",
             "org.apache.spark.sql.execution.datasources.SQLHadoopMapReduceCommitProtocol") \
        .set("spark.jars", obterJars()) \
        .set("spark.executor.memory", "3g") \
        .set("spark.driver.memory", "3g") \
        .set("spark.cores.max", "20") \
        # .set("spark.hadoop.fs.s3a.committer.name", "magic") \
        # .set("spark.hadoop.fs.s3a.bucket.all.committer.magic.enabled", "true")


sc = pyspark.SparkContext(conf=conf).getOrCreate()
sc.setLogLevel("WARN")
client_spark = SparkSession(sc)
client_spark

/tmp/spark-jars/slf4j-reload4j-2.0.6.jar
/tmp/spark-jars/mysql-connector-j-8.1.0.jar
/tmp/spark-jars/aws-java-sdk-dynamodb-1.12.400.jar
/tmp/spark-jars/hadoop-aws-3.3.4.jar
/tmp/spark-jars/httpclient5-5.2.1.jar
/tmp/spark-jars/aws-java-sdk-kms-1.12.400.jar
/tmp/spark-jars/aws-java-sdk-s3-1.12.400.jar
/tmp/spark-jars/postgresql-42.6.0.jar
/tmp/spark-jars/spark-hadoop-cloud_2.13-3.3.1.jar
/tmp/spark-jars/aws-java-sdk-1.12.400.jar
/tmp/spark-jars/mssql-jdbc-12.4.1.jre11.jar
/tmp/spark-jars/aws-java-sdk-core-1.12.400.jar
/tmp/spark-jars/spark-mssql-connector_2.12_3.0-1.0.0-alpha.jar
/tmp/spark-jars/joda-time-2.12.2.jar
/tmp/spark-jars/slf4j-api-2.0.6.jar


23/10/02 23:48:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
def salvarDataFrameBancoDados(df, bancoDados:str, tabela: str, modoEscrita="overwrite"):
    df.write \
        .format("jdbc") \
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
        .option("url", f"jdbc:sqlserver://sql-server:1433;databaseName={bancoDados};") \
        .option("dbtable", tabela) \
        .option("user", "SA") \
        .option("password", "SqlServer2019!") \
        .option("encrypt",False) \
        .mode(modoEscrita) \
        .save()
    

    logging.info(f"{df.count()} registros salvos na tabela {tabela}]")

In [6]:
def getDataframeFromSql(sparkSession: SparkSession, bancoDados: str, sql: str):
    df = sparkSession.read \
        .format("jdbc") \
        .option("url",  f"jdbc:sqlserver://sql-server:1433;databaseName={bancoDados};") \
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
        .option("user", "SA") \
        .option("password", "SqlServer2019!") \
        .option("query", sql) \
        .option("encrypt",False) \
        .load()

    return df

In [7]:
def salvarDimensaoTempo():
    SQL_RANGE_DATAS = """
    SELECT 
        min(dt_atracacao) data_inicial,
        max(dt_atracacao) data_final
    FROM atracacao
    """
    dfRangeDatas = getDataframeFromSql(client_spark, "antaq", SQL_RANGE_DATAS)
    dfDatas = dfRangeDatas.selectExpr("explode(sequence(data_inicial,data_final)) as data")
    dfDimTempo = dfDatas \
        .withColumn("ano", date_format(dfDatas.data, "yyyy")) \
        .withColumn("mes", date_format(dfDatas.data, "MM")) \
        .withColumn("dia", date_format(dfDatas.data, "dd")) \
        .withColumn("ano_mes", date_format(dfDatas.data, "yyyy-MM")) \
        .withColumn("trimestre", date_format(dfDatas.data, "yyyy-Q"))
    logging.info(f"Total registros de tempo: {dfDimTempo.count()}")
    print("===========================================")
    dfDimTempo.show(5)
    print("===========================================")
    logging.info(f"Criando tabela dimensão tempo no DW")
    salvarDataFrameBancoDados(dfDimTempo, "antaq_dw", "dim_tempo")

In [8]:
salvarDimensaoTempo()

+-------------------+----+---+---+-------+---------+
|               data| ano|mes|dia|ano_mes|trimestre|
+-------------------+----+---+---+-------+---------+
|2000-10-11 14:00:00|2000| 10| 11|2000-10|   2000-4|
|2000-10-12 14:00:00|2000| 10| 12|2000-10|   2000-4|
|2000-10-13 14:00:00|2000| 10| 13|2000-10|   2000-4|
|2000-10-14 14:00:00|2000| 10| 14|2000-10|   2000-4|
|2000-10-15 14:00:00|2000| 10| 15|2000-10|   2000-4|
+-------------------+----+---+---+-------+---------+
only showing top 5 rows



In [9]:
def salvarDimensaoPortoInformante():
    SQL_DIM_PORTO = """
        SELECT
        	cod_porto_informante,cod_uf
        FROM atracacao 
        where 1=1
        group by cod_porto_informante, cod_uf   
    """
    dfDimPorto = getDataframeFromSql(client_spark, "antaq", SQL_DIM_PORTO)
    logging.info(f"Total registros de porto: {dfDimPorto.count()}")
    print("===========================================")
    dfDimPorto.show(5)
    print("===========================================")
    logging.info(f"Criando tabela dimensão porto no DW")
    salvarDataFrameBancoDados(dfDimPorto, "antaq_dw", "dim_porto")

In [10]:
salvarDimensaoPortoInformante()

+--------------------+------+
|cod_porto_informante|cod_uf|
+--------------------+------+
|             BRAM013|    AM|
|             BRAM011|    AM|
|             BRAM001|  null|
|               BRMCZ|    AL|
|               BRRIO|  null|
+--------------------+------+
only showing top 5 rows



In [11]:
def salvarDimensaoLocalizacao():
    SQL_DIM_LOCALIZACAO = """
        SELECT
        	municipio,cod_uf,nome_uf,regiao_hidrografica
        FROM antaq.dbo.atracacao 
        where 1=1
        group by nome_uf , cod_uf,municipio ,regiao_hidrografica  
    """
    dfDimLocalizacao = getDataframeFromSql(client_spark, "antaq", SQL_DIM_LOCALIZACAO)
    logging.info(f"Total registros de proto: {dfDimLocalizacao.count()}")
    print("===========================================")
    dfDimLocalizacao.show(5)
    print("===========================================")
    logging.info(f"Criando tabela dimensão localização no DW")
    salvarDataFrameBancoDados(dfDimLocalizacao, "antaq_dw", "dim_localizacao")

In [ ]:
salvarDimensaoLocalizacao()

In [ ]:
def salvarDimensaoBerco():
    SQL_DIM_BERCO = """
        SELECT
        	id_berco,nome_berco
        FROM antaq.dbo.atracacao 
        where 1=1
        group by id_berco ,nome_berco  
    """
    dfDimBerco = getDataframeFromSql(client_spark, "antaq", SQL_DIM_BERCO)
    logging.info(f"Total registros de berco: {dfDimBerco.count()}")
    print("===========================================")
    dfDimBerco.show(5)
    print("===========================================")
    logging.info(f"Criando tabela dimensão porto no DW")
    salvarDataFrameBancoDados(dfDimBerco, "antaq_dw", "dim_berco")

In [ ]:
salvarDimensaoBerco()

In [ ]:
def salvarFatoAtracacao():
    SQL_FATO_ATRACACOES = """
        SELECT 
            id_atracacao, 
            cod_porto_informante, 
            nome_porto_informante, 
            alias_porto_informante,
            id_berco, 
            -- nome_berco, 
            complexo_portuario, 
            tipo_autoridade_portuaria, 
            dt_chegada, 
            dt_atracacao, 
            dt_inicio_operacao, 
            dt_fim_operacao, 
            dt_desatracacao, 
            CASE tipo_operacao
                WHEN '1' THEN 'Movimentacão de Carga'
                WHEN '2' THEN 'Passageiro'
                WHEN '3' THEN 'Apoio'
                WHEN '4' THEN 'Marinha'
                WHEN '5' THEN 'Abastecimento'
                WHEN '6' THEN 'Reparo/Manutenção'
                WHEN '7' THEN 'Misto'
                WHEN '8' THEN 'Retirada de Resíduos'
                ELSE tipo_operacao
            END tipo_operacao, 
            CASE tipo_navegacao 
                WHEN '1' THEN 'Navegação Interior'
                WHEN '2' THEN 'Apoio Portuário'
                WHEN '3' THEN 'Cabotagem'
                WHEN '4' THEN 'Apoio Marítimo'
                WHEN '5' THEN 'Longo Curso'
                ELSE tipo_navegacao
            END tipo_navegacao, 
            CASE nacionalidade_armador
                WHEN '1' THEN 'Brasileira'
                WHEN '2' THEN 'Estrangeira'
                ELSE nacionalidade_armador
            END nacionalidade_armador, 
            CASE contabiliza_movimentacao
                WHEN '1' THEN 'S'
                ELSE 'N'
            END contabiliza_movimentacao, 
            terminal, 
            municipio, 
            -- nome_uf, 
            -- cod_uf, 
            -- regiao_hidrografica, 
            -- cod_capitania, 
            cod_IMO, 
            tempo_espera_atracacao, 
            tempo_espera_operacao, 
            tempo_operacao, 
            tempo_espera_desatracacao, 
            tempo_atracado, 
            tempo_estadia
        FROM atracacao
    """
    dfFatoAtracacao = getDataframeFromSql(client_spark, "antaq", SQL_FATO_ATRACACOES)
    logging.info(f"Total registros de atracacoes: {dfFatoAtracacao.count()}")
    print("===========================================")
    dfFatoAtracacao.show(5)
    print("===========================================")
    logging.info(f"Criando tabela fato atracacoes no DW")
    salvarDataFrameBancoDados(dfFatoAtracacao, "antaq_dw", "atracacoes_fato")


In [ ]:
salvarFatoAtracacao()

In [ ]:
def salvarFatoCarga():
    SQL_FATO_CARGA = """
        SELECT 
            id_carga, 
            c.id_atracacao, 
            cod_porto_origem, 
            cod_porto_destino, 
            cod_mercadoria, 
            c.tipo_operacao, 
            tipo_acondicionamento, 
            estado_container, 
            c.tipo_navegacao, 
            flag_autorizacao, 
            flag_cabotagem, 
            flag_cabotagem_movimentacao, 
            tamanho_container, 
            flag_longo_curso, 
            tipo_operacao_carga, 
            flag_offshore, 
            flag_via_interior, 
            percurso_vias_interiores, 
            percurso_interiores, 
            unica_natureza, 
            unico_capitulo, 
            unica_mercadoria, 
            natureza_carga, 
            sentido_operacao, 
            qtd_TEU_movimentacao, 
            qtd_unidades_movimentadas, 
            peso_bruto, 
            peso_liquido ,
            dt_atracacao,
            cod_porto_informante cod_porto_atracacao,
            municipio,
            YEAR(dt_atracacao) ano_atracacao, 
            MONTH(dt_atracacao) mes_atracacao,
            cod_uf_atracacao
        FROM carga c left join atracacao a on c.id_atracacao=a.id_atracacao
    """
    dfFatoCarga = getDataframeFromSql(client_spark, "antaq", SQL_FATO_CARGA)
    logging.info(f"Total registros de atracacoes: {dfFatoCarga.count()}")
    print("===========================================")
    dfFatoCarga.show(5)
    print("===========================================")
    logging.info(f"Criando tabela fato atracacoes no DW")
    salvarDataFrameBancoDados(dfFatoCarga, "antaq_dw", "cargas_fato")

In [ ]:
salvarFatoCarga()

In [ ]:
client_spark.stop()